# Day 24: Lobby Layout

Your raft makes it to the tropical island; it turns out that the small crab was an excellent navigator. You make your way to the resort.

As you enter the lobby, you discover a small problem: the floor is being renovated. You can't even reach the check-in desk until they've finished installing the new tile floor.

The tiles are all hexagonal; they need to be arranged in a hex grid with a very specific color pattern. Not in the mood to wait, you offer to help figure out the pattern.

The tiles are all white on one side and black on the other. They start with the white side facing up. The lobby is large enough to fit whatever pattern might need to appear there.

A member of the renovation crew gives you a list of the tiles that need to be flipped over (your puzzle input). Each line in the list identifies a single tile that needs to be flipped by giving a series of steps starting from a reference tile in the very center of the room. (Every line starts from the same reference tile.)

Because the tiles are hexagonal, every tile has six neighbors: east, southeast, southwest, west, northwest, and northeast. These directions are given in your list, respectively, as e, se, sw, w, nw, and ne. A tile is identified by a series of these directions with no delimiters; for example, esenee identifies the tile you land on if you start at the reference tile and then move one tile east, one tile southeast, one tile northeast, and one tile east.

Each time a tile is identified, it flips from white to black or from black to white. Tiles might be flipped more than once. For example, a line like esew flips a tile immediately adjacent to the reference tile, and a line like nwwswee flips the reference tile itself.

Here is a larger example:

```text
sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew
```

In the above example, 10 tiles are flipped once (to black), and 5 more are flipped twice (to black, then back to white). After all of these instructions have been followed, a total of 10 tiles are black.

Go through the renovation crew's list and determine which tiles they need to flip. After all of the instructions have been followed, how many tiles are left with the black side up?

In [1]:
# Python imports
from collections import Counter
from pathlib import Path
from typing import List, Set, Tuple

We use two functions to parse the input data: `load_data()` loops over each line in the file, and `read_path()` recognises whether the first two characters are a known two-character direction (`nw`, `ne`, `sw`, or `se`) - in which case we consume that direction - or a single-character direction (`w` or `e`), which we consume if we find.

Each path is returned as a list of directions to move, and the file contents are returned as a list of these paths.

In [2]:
def read_path(line: str) -> List[str]:
    """Convert string of directions to list of directions
    
    :param line:  concatenated string of directions
    """
    path = []  # individual directions in path
    while len(line):
        # Consume valid two-character directions
        if line[:2] in ("nw", "ne", "sw", "se"):
            path.append(line[:2])
            line = line[2:]
        elif line[:1] in ("w", "e"):  # or valid one-character directions
            path.append(line[0])
            line = line[1:]
        else:
            raise ValueError(f"Expected one of e, w, ne, nw, se, sw; got {line[:2]}")
    return path
    

def load_data(fpath: str) -> List[List[str]]:
    paths = []  # individual paths
    with Path(fpath).open("r") as ifh:
        for line in [_.strip() for _ in ifh.readlines()]:
            paths.append(read_path(line))  # parse path
    return paths

We move on a hex grid of coordinates, assuming an origin at (0, 0). With this system, moves on the y-axis are like any other 2D grid (y is incremented by 1 when moving N, decremented by 1 when moving S), but moves on the y-axis depend on whether the row is odd- or even-numbered, and whether the move involves a change in the y-axis.

On all rows, moving E increments x by 1, moving W decrements x by 1.

On even rows, moving NE or SE increments x by 1, moving NW or SW *does not change x*.

However, on odd rows moving NW or SW decrements x by 1, moving NE or SE *does not change x*.

The `get_final_location()` function steps through each move in a passed `path`, starting from the origin (by default), and gives the location of the final tile the path lands on.

In [3]:
def get_final_location(path: List[str], start: Tuple[int, int]=(0, 0)) -> Tuple[int, int]:
    """Return final location of path on hex grid, starting from start
    
    :param path:  path as list of directions
    :param start:  starting location (x, y)
    
    The subtlety here is that while movements on a hex grid are "normal" in
    the y-axis, we need to change the x-axis movements depending on whether
    the current y-axis co-ordinate is odd or even.
    """
    # Each hex cell has co-ordinate (x, y)
    curx, cury = start[0], start[1]
    for step in path:
        if step == "w":
            curx -= 1
        elif step == "e":
            curx += 1
        elif step == "nw":
            if cury % 2:  # odd rows, decrement x
                curx -= 1
            cury += 1
        elif step == "ne":
            if cury % 2 == 0:  # even rows, increment x
                curx += 1
            cury += 1
        elif step == "sw":
            if cury % 2:  # odd rows, decrement x
                curx -= 1
            cury -= 1
        elif step == "se":
            if cury % 2 == 0:  # even rows, increment x
                curx += 1
            cury -= 1
    return (curx, cury)

We test out with a couple of examples from the puzzle text:

In [4]:
print(get_final_location(read_path("esew")))
print(get_final_location(read_path("nwwswee")))

(1, -1)
(0, 0)


Now we try the test data. We read in the paths from file, and then collect the final location in the `flips` variable. We use the Python `Counter` object to count the number of times each tile is landed on, and report the number of tiles that are landed on an odd number of times (these are black; even flips returns them to white).

In [5]:
paths = load_data("day24_test.txt")
flips = [get_final_location(path) for path in paths]
sum([_ for _ in Counter(flips).values() if _ % 2])  # how many tiles were flipped an odd number of times (black)

10

Doing the same for the the real puzzle data:

In [6]:
paths = load_data("day24_data.txt")
flips = [get_final_location(path) for path in paths]
sum([_ for _ in Counter(flips).values() if _ % 2])  # how many tiles were flipped an odd number of times (black)

538

## Part Two

The tile floor in the lobby is meant to be a living art exhibit. Every day, the tiles are all flipped according to the following rules:

    Any black tile with zero or more than 2 black tiles immediately adjacent to it is flipped to white.
    Any white tile with exactly 2 black tiles immediately adjacent to it is flipped to black.

Here, tiles immediately adjacent means the six tiles directly touching the tile in question.

The rules are applied simultaneously to every tile; put another way, it is first determined which tiles need to be flipped, then they are all flipped at the same time.

In the above example, the number of black tiles that are facing up after the given number of days has passed is as follows:

```text
Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37

Day 20: 132
Day 30: 259
Day 40: 406
Day 50: 566
Day 60: 788
Day 70: 1106
Day 80: 1373
Day 90: 1844
Day 100: 2208
```

After executing this process a total of 100 times, there would be 2208 black tiles facing up.

How many tiles will be black after 100 days?

This is a variant of the Game of Life, but on a hex grid. We use the same approach as in an earlier puzzle, storing black tiles in a set of co-ordinates, and checking these tiles (and their white neighbours) against the rules of the game. A *copy* of the input tile set is updated on each turn - so we can update synchronously - and the neighbouring tile identification (in `get_neighbours()`) is updated to account for the hex field.

In [7]:
def get_neighbours(tile: Tuple[int, int]) -> List[Tuple[int, int]]:
    """Return coordinates of tile neighbours on hexgrid
    
    :param tile:  single tile co-ordinates on the hex grid
    
    As before, the subtlety is that the neighbour co-ordinate
    calculations depend on whether the central tile is on the
    odd- or even-numbered y-axis row.
    """
    curx, cury = tile
    if cury % 2:  # odd row, decrement x for NW, SW
        return {(curx - 1, cury), (curx + 1, cury),
                (curx - 1, cury + 1), (curx, cury + 1), 
                (curx - 1, cury - 1), (curx, cury - 1)}
    else:  # even row, increment x for NE, SE
        return {(curx - 1, cury), (curx + 1, cury),
                (curx, cury + 1), (curx + 1, cury + 1), 
                (curx, cury - 1), (curx + 1, cury - 1)}

def hex_life(black_tiles: Set[Tuple[int, int]]) -> Set[Tuple[int, int]]:
    """Return the next state in a hex grid Game of Life
    
    :param black_tiles:  current state in hex grid Game of Life
    
    For efficiency, we need only check the current set of black tiles,
    and their white tile neighbours; no other tiles would need updating
    """
    next_step = black_tiles.copy()  # copy for synchronous update
    
    for tile in black_tiles:
        # Check neighbouring black tile count
        nbrs = get_neighbours(tile)
        blk_nbrs = len(nbrs.intersection(black_tiles))
        if blk_nbrs > 2 or blk_nbrs == 0:
            next_step.remove(tile)
        # Check neighbouring white tiles for potential update
        for nbr in [_ for _ in nbrs if _ not in black_tiles]:
            if len(black_tiles.intersection(get_neighbours(nbr))) == 2:
                next_step.add(nbr)
                
    return next_step


Now we load the paths from file (`load_data()`), get the initial setup of tiles (`get_final_location()`) for each of the paths, and get a set of black tiles in the `black_tiles` variable. Then we implement the `hex_life()` rules 100 times, and count the number of black tiles on the final day, for the test data:

In [8]:
paths = load_data("day24_test.txt")
flips = [get_final_location(path) for path in paths]
black_tiles = {tile for (tile, cnt) in Counter(flips).items() if cnt % 2}
for day in range(100):
    black_tiles = hex_life(black_tiles)
print(f"Day {day + 1}: {len(black_tiles)}")

Day 100: 2208


Doing the same for the real data:

In [9]:
paths = load_data("day24_data.txt")
flips = [get_final_location(path) for path in paths]
black_tiles = {tile for (tile, cnt) in Counter(flips).items() if cnt % 2}
for day in range(100):
    black_tiles = hex_life(black_tiles)
print(f"Day {day + 1}: {len(black_tiles)}")

Day 100: 4259
